In [1]:
# PROJECT: Inventory Optimization & Demand Forecasting Decision System
# NOTEBOOK: 03 - Inventory Optimization

# OBJECTIVE:
# - Convert forecasted demand into inventory decisions
# - Apply EOQ, Safety Stock, and Reorder Point logic
# - Evaluate service level scenarios
# - Generate decision-ready outputs

In [2]:
# IMPORT REQUIRED LIBRARIES
import pandas as pd
import numpy as np

In [3]:
# LOAD FORECAST OUTPUT (FROM NOTEBOOK 2)
forecast_df = pd.read_csv('forecasted_demand.csv')

forecast_df['Month'] = pd.to_datetime(forecast_df['Month'])

forecast_df

,sku_group,Month,Forecasted_Demand
0,ALL_SKUS,2015-01-01,7077.383677
1,ALL_SKUS,2015-02-01,6994.915298
2,ALL_SKUS,2015-03-01,6979.302858
3,ALL_SKUS,2015-04-01,6976.347200
4,ALL_SKUS,2015-05-01,6975.787652
5,ALL_SKUS,2015-06-01,6975.681722


In [4]:
# DEFINE INVENTORY INPUT PARAMETERS
# Inventory parameters (assumptions documented clearly)

lead_time_months = 1
holding_cost_per_unit = 2.0      # cost per unit per month
ordering_cost = 500.0            # cost per order
stockout_cost_per_unit = 10.0    # penalty cost

In [5]:
# CALCULATE AVERAGE DEMAND & VARIABILITY
avg_demand = forecast_df['Forecasted_Demand'].mean()
demand_std = forecast_df['Forecasted_Demand'].std()

avg_demand, demand_std

(np.float64(6996.569734420301), 40.27140540100847)

In [6]:
# ECONOMIC ORDER QUANTITY (EOQ)
EOQ = np.sqrt((2 * avg_demand * ordering_cost) / holding_cost_per_unit)

EOQ

np.float64(1870.3702486968054)

In [7]:
# SAFETY STOCK CALCULATION
# Z-scores for service levels
service_levels = {
    '90%': 1.28,
    '95%': 1.65,
    '99%': 2.33
}

safety_stock = {
    level: z * demand_std * np.sqrt(lead_time_months)
    for level, z in service_levels.items()
}

safety_stock

{'90%': np.float64(51.54739891329085),
 '95%': np.float64(66.44781891166397),
 '99%': np.float64(93.83237458434974)}

In [8]:
# REORDER POINT (ROP)
ROP = {
    level: (avg_demand * lead_time_months) + ss
    for level, ss in safety_stock.items()
}

ROP

{'90%': np.float64(7048.1171333335915),
 '95%': np.float64(7063.017553331965),
 '99%': np.float64(7090.402109004651)}

In [9]:
# SERVICE LEVEL SCENARIO ANALYSIS
scenario_results = []

for level in service_levels:
    scenario_results.append({
        'Service_Level': level,
        'EOQ': EOQ,
        'Safety_Stock': safety_stock[level],
        'Reorder_Point': ROP[level]
    })

scenario_df = pd.DataFrame(scenario_results)

scenario_df

,Service_Level,EOQ,Safety_Stock,Reorder_Point
0,90%,1870.370249,51.547399,7048.117133
1,95%,1870.370249,66.447819,7063.017553
2,99%,1870.370249,93.832375,7090.402109


In [10]:
# FINAL DECISION OUTPUT
decision_output = scenario_df.copy()
decision_output['sku_group'] = 'ALL_SKUS'

decision_output = decision_output[
    ['sku_group', 'Service_Level', 'EOQ', 'Safety_Stock', 'Reorder_Point']
]

decision_output

,sku_group,Service_Level,EOQ,Safety_Stock,Reorder_Point
0,ALL_SKUS,90%,1870.370249,51.547399,7048.117133
1,ALL_SKUS,95%,1870.370249,66.447819,7063.017553
2,ALL_SKUS,99%,1870.370249,93.832375,7090.402109


In [11]:
# SAVE OUTPUT FOR POWER BI & REPORT
decision_output.to_csv('inventory_decision_output.csv', index=False)

print("Inventory optimization decisions saved successfully.")

Inventory optimization decisions saved successfully.
